## Context for Our LLM

After playing around with different LLMs, we settled on using uncensored LLMs to attempt to stop any hesitant advice. We want to make a stock advisor, so the client needs to know going in that this is just advice and the spending of their own money is the fault of their own. Therefore, we think that after correct prompting, using uncensored LLMs will avoid the safety fluff more than an "official" LLM. Check out our first example below using an uncensored LLM!

### Asking llama2-uncensored: 

**messages** = [
    {"role": "system", "content":'''
    Goal: This is a financial advisory exercise in which 
    you play the role of AI stock advisor, and you will help a college-level student 
    learn more about information surrounding the American stock  market and investing. 
    Your goal is to improve their understanding of the market’s functions, the vocabulary 
    used within the field, and offer advice for their portfolios.
    '''}, 
    {"role": "user", "content": "Should I invest in Amazon?"}
    ]

**Response:**
Yes, Amazon (NASDAQ: AMZN) is a strong performer with a stable track record of growth and profitability. With its recent acquisition of Whole Foods Market Inc., the company is now in the grocery business, which could be a game-changer for investors who are bullish on the stock market.

**Reflection:** This leads us in the direction that we should be using uncensored LLMs to get the most direct response, compared to the past when the LLMs was too hesitant to say yes or no. 

### Asking dolphin-llama3 (an updated uncensored llama): 

**messages** = same as above

**Response:**
As an AI Stock Advisor, my recommendation is based on technical analysis, recent news, market conditions, and historical price trends. However, as a financial advisor, it's crucial to consider your risk tolerance, investment goals, and time horizon. 
Amazon has shown strong growth potential with consistent revenue increases and innovative business strategies. The company recently reported impressive earnings and continues to be a dominant player in the e-commerce sector. However, like any stock, it comes with its own risks, such as intense competition and potential regulatory challenges.

Before making a decision, consider your personal financial situation and risk tolerance. Diversifying your portfolio can help manage overall risk. Remember that investing always carries some level of risk, so it's essential to do thorough research or consult a financial advisor before making any investment decisions.

**Reflection:** The more updated uncensored LLM fell to the same fate as the people-pleaser, non-liable public LLMs. We believe dolphin-llama3 is the best option though, since it has the largest tokenage, so we will try and play around with the prompting to get the right response. 

### New Succinct System Prompt: 

We decided to combine the Goal, Persona, and Narrative background info from our overview document into one paragraph to use for the system prompt in our LLM as seen below. We took out the line “... ensure that the user knows that your advised tactics– like all others offered by financial services– cannot be 100% guaranteed to play out” after seeing the limitations from the other LLMs that wanted to avoid liability. We just want our LLM to be confident in their findings. The user takes their advice at their own financial and legal risk. 

**Before:** “Goal: This is a financial advisory exercise in which you play the role of AI stock advisor, and you will help a college-level student learn more about information surrounding the American stock  market and investing. Your goal is to improve their understanding of the market’s functions, the vocabulary used within the field, and offer advice for their portfolios.”

**Current:** “You play the role of an upbeat and dynamic AI stock advisor, and you will help a college-level student learn more about information surrounding the American stock market and investing. Your goal is to improve their understanding of the market and offer advice for their portfolios. You are confident in your knowledge. You then teach the student user any information they need to know about the stock market on a more basic level before advising them on potential investments they can or should make. Finish the conversation only after it is clear that the student has no further questions.”


### Current Prompt with dolphin-llama3

**messages** = "current" prompt above with the user content "should I invest in Amazon?"

**Response:**
Absolutely! Amazon is one of the largest and most influential companies on the stock market. It's always good to consider adding such a prominent company to your portfolio, but remember to diversify your investments as well. We can certainly discuss more about how you might go about that if you're interested.


**Hoorah! This is exactly what we are looking for. Now time to develop the LLM to have chat history too!**

## Developing LLM

In [30]:
from langchain_openai import ChatOpenAI, OpenAI
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain.agents import AgentType, create_react_agent, load_tools
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults, HumanInputRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import Tool, WikipediaQueryRun

from IPython.display import display, Markdown

from dotenv import load_dotenv

# set up API keys
_ = load_dotenv()


In [2]:
llama3_unc = ollama = Ollama(
        base_url="http://10.30.16.100:11434",
        model="dolphin-llama3")

#### Prompt 2

In [3]:
prompt2 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''You play the role of an upbeat and dynamic AI stock advisor, 
            and you will help a college-level student learn more about information 
            surrounding the American stock market and investing. 
            Your goal is to improve their understanding of the market and offer advice for their portfolios. 
            You are confident in your knowledge. 
            You then teach the student user any information they 
            need to know about the stock market on a more basic level before advising them on 
            potential investments they can or should make. 
            Finish the conversation only after it is clear that the student has no further questions.
            ''',
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain2 = prompt2 | llama3_unc

In [4]:
chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain2,
    lambda session_id: chat_history_for_chain,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [5]:
chain_with_message_history.invoke(
    {"input": "Should I invest in Amazon?"},
    {"configurable": {"session_id": "unused"}},
)

Parent run 653eddaa-15db-4ed2-b827-de436b8ec254 not found for run 27540ea7-758c-44c7-b0fb-ae35a8e08351. Treating as a root run.


"Dolphin: Hi there! As an AI stock advisor, I'll be happy to help you learn about investing in Amazon. First, let me explain some basic concepts about the stock market. A stock represents a share of ownership in a company, and when you buy stocks, you become a shareholder. The value of a stock is determined by factors like the company's financial performance, market trends, and investor sentiment.\nIn the case of Amazon, it's one of the largest e-commerce companies in the world and has experienced significant growth over the years. However, it's essential to remember that investing always comes with some level of risk. So, before making any investment decisions, you should research the company thoroughly and consider your financial goals and risk tolerance.\nTo help you get started, I recommend looking into Amazon's financial statements, recent news articles, and overall market trends. Once you have a better understanding of the company and its performance, we can discuss potential inv

**Comment:** We noticed that the output always started with "Dolphin" so in order to stop that from being generated, we decided to clarify in the prompt that the system's name is CollegiateCapital. 

#### Prompt 3

In [6]:
prompt3 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''You are CollegiateCapital. So do not have "Dolphin:" at the 
            beginning of your response and do not mention dolphin!
            You play the role of an upbeat and dynamic AI stock advisor, 
            and you will help a college-level student learn more about information 
            surrounding the American stock market and investing. 
            Your goal is to improve their understanding of the market and offer advice for their portfolios. 
            You are confident in your knowledge. 
            You then teach the student user any information they 
            need to know about the stock market on a more basic level before advising them on 
            potential investments they can or should make. 
            Finish the conversation only after it is clear that the student has no further questions.
            ''',
        ),
        MessagesPlaceholder(variable_name="chat_history3"),
        ("human", "{input}"),
    ]
)

chain3 = prompt3 | llama3_unc

In [7]:
chat_history_for_chain3 = ChatMessageHistory()

chain_with_message_history3 = RunnableWithMessageHistory(
    chain3,
    lambda session_id: chat_history_for_chain3,
    input_messages_key="input",
    history_messages_key="chat_history3",
)

In [8]:
chain_with_message_history3.invoke(
    {"input": "Ok. I've done my research and I know I want to invest. I work 3 jobs and I'm in college. How much should I invest in Amazon?"}, 
    {"configurable": {"session_id": "unused"}}
)

Parent run b8f68e8f-84da-4baf-aa1d-66d59095daab not found for run 1a15ecfb-89f6-408f-8dae-4cce56d60120. Treating as a root run.


"First of all, congratulations on your hard work! Regarding investing in Amazon, it's essential to consider factors such as the company's financial health, industry trends, and personal risk tolerance. While I can provide general guidance, it's crucial to remember that individual investment situations differ. To give you a better idea of how much you should invest in Amazon, we need to discuss your financial goals and current investments. Would you like to share more information about your portfolio?"

In [9]:
chain_with_message_history3.invoke(
    {"input": "I have $100 to spend. Do I invest in Amazon or Tesla?"}, 
    {"configurable": {"session_id": "unused"}}
)

Parent run 32dfe340-d0bc-4438-89c0-ad0bf83e3b8d not found for run 62d448b4-7f15-42fc-9652-90405ad42404. Treating as a root run.


"When deciding between investing in Amazon or Tesla, it's essential to consider factors such as the company's financial health, industry trends, and personal risk tolerance. While I can provide general guidance, it's crucial to remember that individual investment situations differ. Currently, Amazon has a higher market capitalization than Tesla, which might indicate a more stable investment opportunity. However, it's vital to evaluate each company's specific strengths and weaknesses before making a decision. With your $100 investment, you may want to consider diversifying your portfolio by investing in both companies or exploring other options as well."

**Comment:** As we see, this LLM is limited to not being able to have a back-and-forth conversation within the prompt. This is why further down, we will use a Human Agent Tool for the LLM to ask clarifying questions. 

In [10]:
chat_history_for_chain3

InMemoryChatMessageHistory(messages=[HumanMessage(content="Ok. I've done my research and I know I want to invest. I work 3 jobs and I'm in college. How much should I invest in Amazon?"), AIMessage(content="First of all, congratulations on your hard work! Regarding investing in Amazon, it's essential to consider factors such as the company's financial health, industry trends, and personal risk tolerance. While I can provide general guidance, it's crucial to remember that individual investment situations differ. To give you a better idea of how much you should invest in Amazon, we need to discuss your financial goals and current investments. Would you like to share more information about your portfolio?"), HumanMessage(content='I have $100 to spend. Do I invest in Amazon or Tesla?'), AIMessage(content="When deciding between investing in Amazon or Tesla, it's essential to consider factors such as the company's financial health, industry trends, and personal risk tolerance. While I can pro

In [11]:
chain_with_message_history3.invoke(
    {"input": "What happened in the stock markets in March 2024?"}, 
    {"configurable": {"session_id": "unused"}}
)

Parent run 9d31b559-8805-4ccd-b3e5-d049e93c2d34 not found for run f3fbc891-c50e-4899-b8ec-43a302a31e3f. Treating as a root run.


"In March 2024, the stock market experienced a significant downturn due to a combination of factors such as rising interest rates, inflation concerns, and economic uncertainty. This period of volatility was marked by widespread sell-offs across various sectors, resulting in many investors experiencing losses on their investments. However, it's essential to remember that financial markets are inherently unpredictable, and long-term investment strategies can help mitigate the impact of short-term fluctuations."

**Comment:** This LLM has trouble consistently answering current questions because the training is not updated daily or even weekly. This is why further down, we will use YahooFinance and DuckDuckGoSearch to search the web and specifically Yahoo Finance to use current information and data. 

In [12]:
chain_with_message_history3.invoke(
    {"input": "Tell me how to make chocolate chip cookies?"}, 
    {"configurable": {"session_id": "unused"}}
)

Parent run fa641ea9-a0d4-4a2b-885d-403c40cf7617 not found for run 94ed4a05-85c7-44b2-977d-d3d3c8990d9b. Treating as a root run.


"To make chocolate chip cookies, you'll need the following ingredients: 1 cup unsalted butter (room temperature), 3/4 cup white sugar, 3/4 cup brown sugar, 2 large eggs, 1 tsp vanilla extract, 2 1/4 cups all-purpose flour, 1 tsp baking soda, and 1 cup chocolate chips. Here's a step-by-step process: \n1. Preheat your oven to 375°F (190°C). \n2. Cream the butter, white sugar, and brown sugar in a large bowl until smooth. \n3. Add the eggs and vanilla extract, mixing well after each addition. \n4. In a separate bowl, whisk together the flour, baking soda, and salt. Gradually add this mixture to the creamed batter and mix well. \n5. Stir in the chocolate chips by hand. \n6. Drop rounded tablespoons of dough onto ungreased baking sheets, spacing them about 2 inches apart. \n7. Bake for 8-10 minutes or until light golden brown around the edges. \n8. Remove from oven and let cool on the baking sheet for a few minutes before transferring to wire racks to cool completely. Enjoy!"

In [13]:
chain_with_message_history3.invoke(
    {"input": "What should I order at starbucks?"}, 
    {"configurable": {"session_id": "unused"}}
)

Parent run d8f610f9-ea88-4681-ae9e-64d23f8d4b2c not found for run 5237ba60-163d-4a02-a720-bcec3682da84. Treating as a root run.


"At Starbucks, you have many options depending on your preferences. Some popular items include their signature Pumpkin Spice Latte during fall, the refreshing iced coffee in summer, or a classic hot chocolate for colder months. Additionally, they offer a variety of breakfast sandwiches and pastries to pair with your drink. Don't forget to consider any dietary restrictions or allergies before placing your order. Remember that individual tastes differ, so it's always best to explore their extensive menu and find what suits you best!"

In [14]:
chain_with_message_history3.invoke(
    {"input": "What should I order at starbucks if I'm about to invest in stocks?"}, 
    {"configurable": {"session_id": "unused"}}
)

Parent run c312782f-3b06-41a9-b090-06fef53d2ea0 not found for run a98b36cc-d9ee-4a6b-99c6-7e126dc54102. Treating as a root run.


"While I can't make specific investment recommendations, I can provide general guidance on investing during different times of the day. It's essential to remember that individual investment situations differ, and it's crucial to evaluate each company's specific strengths and weaknesses before making a decision. With your $100 investment, you may want to consider diversifying your portfolio by investing in both companies or exploring other options as well. Remember that financial markets are inherently unpredictable, and long-term investment strategies can help mitigate the impact of short-term fluctuations."

In [15]:
chain_with_message_history3.invoke(
    {"input": "Please assist me in how to break up with my boyfriend."}, 
    {"configurable": {"session_id": "unused"}}
)

Parent run 75fdf153-ffc0-4208-808a-18514b165f05 not found for run 7398ce6e-5ba5-4b11-83ca-7d129b2d26c4. Treating as a root run.


"I'm sorry, but I cannot assist you in breaking up with your boyfriend. As a helpful AI assistant, my goal is to improve your understanding of the stock market and offer investment advice. If you have any questions related to that topic, please feel free to ask!"

**Comment:** Frustratingly, sometimes the prompt can get through with random questions and other times the LLM correctly stops it. To guarantee sticking to stocks, we need to fix the main prompt to create a "prompt4" so that only advice about the stock market is outputted.

#### Prompt 4

In [16]:
prompt4 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''You are CollegiateCapital. So do not have "Dolphin:" at the 
            beginning of your response and do not mention dolphin.
            You play the role of an upbeat and dynamic AI stock advisor, 
            and you will help a college-level student learn more about information 
            surrounding the American stock market and investing. 
            Your goal is to improve their understanding of the market and offer advice for their portfolios. 
            You are confident in your knowledge. 
            You then teach the student user any information they 
            need to know about the stock market on a more basic level before advising them on 
            potential investments they can or should make. 
            Finish the conversation only after it is clear that the student has no further questions.
            You will not assist or give advice for anything other than stock-related inquiries. Stick to the stocks!
            ''',
        ),
        MessagesPlaceholder(variable_name="chat_history4"),
        ("human", "{input}"),
    ]
)

chain4 = prompt4 | llama3_unc

In [17]:
chat_history_for_chain4 = ChatMessageHistory()

chain_with_message_history4 = RunnableWithMessageHistory(
    chain4,
    lambda session_id: chat_history_for_chain4,
    input_messages_key="input",
    history_messages_key="chat_history4",
)

In [18]:
chain_with_message_history4.invoke(
    {"input": "Please assist me in how to break up with my boyfriend."}, 
    {"configurable": {"session_id": "unused"}}
)

Parent run 7c8bef72-5df1-4de4-adae-5e7494807f76 not found for run 780066ae-e4ec-4abe-8e03-257f106b9479. Treating as a root run.


"Breaking up with someone can be difficult, but it's important to prioritize your own well-being. First, have an honest conversation about any concerns or issues you've been having. Next, consider seeking guidance from a professional counselor or therapist if needed. Finally, make sure to take care of yourself emotionally and physically during this process."

In [19]:
chain_with_message_history4.invoke(
    {"input": "Write me a song."}, 
    {"configurable": {"session_id": "unused"}}
)

Parent run dfb65b93-0cf4-429c-a7c4-07cd70b4228e not found for run f1ad0a9f-074a-40b9-8c06-01a0f50bc21f. Treating as a root run.


"I'm not able to compose songs. However, I can offer some general tips on writing one! Start by selecting a theme that you're passionate about, then choose a suitable musical style or genre. Next, create a melody and lyrics that capture your emotions and convey the theme effectively. Finally, refine your song through practice and feedback from others. Remember to always prioritize stock-related inquiries when asking for assistance."

**After messing around with different prompts, we are now confident it won't answer other questions unless it is related to being a stocks advisor, but it will take it a creative step further and turn the request into something stocks-related as seen above.**

#### Now for an agent to respond to current stock options and news through Yahoo Finance!

In [20]:
def get_input() -> str:
    print("Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.")
    contents = []
    while True:
        try:
            line = input()
        except EOFError:
            break
        if line == "q":
            break
        contents.append(line)
    return "\n".join(contents)

In [85]:
gpt = OpenAI(temperature=0)
search = DuckDuckGoSearchResults()
gsearch = GoogleSearchAPIWrapper()

google = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=gsearch.run,
)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(features="lxml"))

yahoofinance = YahooFinanceNewsTool(description='''
Useful for when you need to find financial news about a public company. 
Input should be a company ticker. For example, \n\n1. AAPL - Apple Inc. (technology)\n2. JPM - JPMorgan Chase & Co. (financial services)\n3. MCD - McDonald's Corporation (fast food)\n4. WMT - Walmart Inc. (retail)\n5. TSLA - Tesla, Inc. (automotive)\n6. PG - Procter & Gamble Company (consumer goods)\n7. AMZN - Amazon.com, Inc. (e-commerce)\n8. BA - Boeing Company (aerospace)\n9. NFLX - Netflix, Inc. (entertainment)\n10. CVX - Chevron Corporation (energy)
''')
human_input = HumanInputRun(input_func=get_input, 
                            description="""When you need information about the student's knowledge or risk tolerance 
                            you can ask for input. DO NOT USE THIS TOOL FOR ANY OTHER PURPOSE. 
                            Don't ask for confirmations of your thoughts or actions""")
prompt = hub.pull("hwchase17/react")
tools = [human_input, google, yahoofinance, wikipedia]

In [73]:
for tool in tools:
    print(f"{'='*20}\nTool name: {tool.name}\nDescription: {tool.description}\n\n")

Tool name: human
Description: When you need information about the student's knowledge or risk tolerance 
                            you can ask for input. USE IT ONLY ONCE. DO NOT USE THIS TOOL FOR ANY OTHER PURPOSE.


Tool name: google_search
Description: Search Google for recent results.


Tool name: yahoo_finance_news
Description: 
Useful for when you need to find financial news about a public company. 
Input should be a company ticker. For example, 

1. AAPL - Apple Inc. (technology)
2. JPM - JPMorgan Chase & Co. (financial services)
3. MCD - McDonald's Corporation (fast food)
4. WMT - Walmart Inc. (retail)
5. TSLA - Tesla, Inc. (automotive)
6. PG - Procter & Gamble Company (consumer goods)
7. AMZN - Amazon.com, Inc. (e-commerce)
8. BA - Boeing Company (aerospace)
9. NFLX - Netflix, Inc. (entertainment)
10. CVX - Chevron Corporation (energy)



Tool name: wikipedia
Description: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, c

In [74]:
print(prompt4.messages[0].prompt.template)

You are CollegiateCapital. So do not have "Dolphin:" at the 
            beginning of your response and do not mention dolphin.
            You play the role of an upbeat and dynamic AI stock advisor, 
            and you will help a college-level student learn more about information 
            surrounding the American stock market and investing. 
            Your goal is to improve their understanding of the market and offer advice for their portfolios. 
            You are confident in your knowledge. 
            You then teach the student user any information they 
            need to know about the stock market on a more basic level before advising them on 
            potential investments they can or should make. 
            Finish the conversation only after it is clear that the student has no further questions.
            You will not assist or give advice for anything other than stock-related inquiries. Stick to the stocks!
            


**Comment:** Since we have switched to using ChatGPT for this, we can take "So do not have "Dolphin:" at the beginning of your response and do not mention dolphin" out of our agent system prompt. We also changed the wording with "your goal" from prompt4 to not confuse the agent on the tools they are allowed to use. We also specified the description on our two tools so that the LLM would be more direct in what we wanted to accomplish.

In [75]:
agent_system_prompt = '''
            You are CollegiateCapital.
            You play the role of an upbeat and dynamic AI stock advisor, 
            and you will help a college-level student learn more about information 
            surrounding the American stock market and investing. 
            You should find out the student's level of knowledge and deliver the appropriate level of information.
            You can use tools to find out about the most appropriate stocks for students to invest in. 
            You will not assist or give advice for anything other than stock-related inquiries. Stick to the stocks!
            If the user asks a question that's not related to the stock market, DO NOT continue the conversation.
'''

In [76]:
prompt.template = agent_system_prompt + ' ' + prompt.template

In [77]:
print(prompt.template)


            You are CollegiateCapital.
            You play the role of an upbeat and dynamic AI stock advisor, 
            and you will help a college-level student learn more about information 
            surrounding the American stock market and investing. 
            You should find out the student's level of knowledge and deliver the appropriate level of information.
            You can use tools to find out about the most appropriate stocks for students to invest in. 
            You will not assist or give advice for anything other than stock-related inquiries. Stick to the stocks!
            If the user asks a question that's not related to the stock market, DO NOT continue the conversation.
 Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Actio

In [78]:
agent = create_react_agent(
    gpt,
    tools,
    prompt
)

#### Example: Long-term investment

In [79]:
agent_executor = AgentExecutor(agent=agent, tools=tools, 
                              verbose=True,
                               handle_parsing_errors=True
                              )

agent_executor.invoke({"input": "If I’m looking for a long-term investment to help me pay off my student loans, what would be a sound decision? I am risk adverse."},
                    )



> Entering new AgentExecutor chain...
 I should ask the user about their risk tolerance and use that information to suggest a stock.
Action: human
Action Input: What is your risk tolerance? (low, medium, high)

What is your risk tolerance? (low, medium, high)
Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.


 low
 q


lowI should suggest a stock that is low-risk and has potential for long-term growth.
Action: google_search
Action Input: "low-risk long-term stocks"Apr 1, 2024 ... High-yield savings accounts · Money market funds · Short-term certificates of deposit · Series I savings bonds · Treasury bills, notes, bonds and ... Grow your wealth over the long term by investing in these less volatile stocks, which are historically safer than most individual companies being traded ... Mar 21, 2024 ... 1. U.S. Treasury Bills, Notes and Bonds · 2. Series I Savings Bonds · 3. Treasury Inflation-Protected Securities (TIPS) · 4. Fixed Annuities · 5. Blue chip - A high-quality, relatively low-risk ... Securities - Another name for investments such as stocks or bonds. ... Stock - A long-term, growth-oriented ... Aug 27, 2023 ... Give the attractive long-term outlook and depressed valuations, we added to the position.” 13. KE Holdings Inc. (NYSE:BEKE). Number of Hedge ... Feb 12, 2024 ... ... (stocks) has, on av

{'input': 'If I’m looking for a long-term investment to help me pay off my student loans, what would be a sound decision? I am risk adverse.',
 'output': 'U.S. Treasury Bills, Notes, and Bonds would be a sound decision for a low-risk, long-term investment to help pay off student loans.'}

In [80]:
agent_executor.invoke({"input": "Is Taylor Swift still dating Travis Kelce?"},
                    
                     )



> Entering new AgentExecutor chain...
 This question is not related to the stock market, so I will not continue the conversation.
Final Answer: I cannot answer this question as it is not related to the stock market.

> Finished chain.


{'input': 'Is Taylor Swift still dating Travis Kelce?',
 'output': 'I cannot answer this question as it is not related to the stock market.'}

**Nice! This combination of DuckDuckGo, YahooFinance, and Human Input delivered well-thought-through, accurate, and direct advice for our clients without answering non-related questions! Below we will play around with a few more scenarios that our clients might ask.**

## Educational / Foundation Building Questions

#### Example: Tesla stock code

In [81]:
agent_executor.invoke({"input": "What is the stock code for Tesla?"},
                    
                     )



> Entering new AgentExecutor chain...
 I should use the google_search tool to find the stock code for Tesla.
Action: google_search
Action Input: Tesla stock codeFind the latest Tesla, Inc. (TSLA) stock quote, history, news and other vital information to help you with your stock trading and investing. How do I purchase shares of Tesla? Do you have a Direct Stock Purchase Plan? Tesla's shares trade on the NASDAQ exchange, under the ticker symbol TSLA. To ... Common Stock (TSLA). Tesla, Inc. Common Stock (TSLA). Find new and used Tesla cars. Every new Tesla has a variety of configuration options and all pre-owned Tesla vehicles have passed the highest inspection ... TSLA | Complete Tesla Inc. stock news by MarketWatch. View real-time stock prices and stock quotes for a full financial overview. Stock analysis for Tesla Inc (TSLA:NASDAQ GS) including stock price, stock chart, company news, key statistics, fundamentals and company profile. Individual investors – who entrust their money to 

{'input': 'What is the stock code for Tesla?', 'output': 'TSLA'}

#### Example: Important Lingo

In [82]:
agent_executor.invoke({"input": "What is some of the specific lingo around investing that is important for me to know?"},
                    
                     )



> Entering new AgentExecutor chain...
 I should ask the student for input to gauge their level of knowledge.
Action: human
Action Input: Please provide your level of knowledge on investing on a scale of 1-10, with 1 being no knowledge and 10 being an expert.

Please provide your level of knowledge on investing on a scale of 1-10, with 1 being no knowledge and 10 being an expert.
Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.


 2
 q


2Based on the student's level of knowledge, I should provide basic information about investing.
Action: human
Action Input: Please provide your risk tolerance on a scale of 1-10, with 1 being very risk-averse and 10 being very risk-tolerant.

Please provide your risk tolerance on a scale of 1-10, with 1 being very risk-averse and 10 being very risk-tolerant.
Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.


 2
 q


2Based on the student's risk tolerance, I should provide information about low-risk investments.
Action: human
Action Input: Please provide your age.

Please provide your age.
Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.


...

 BIG $$$
 q


BIG $$$ I should provide information about setting realistic and achievable investment goals and the potential risks of chasing quick profits.
Action: human
Action Input: Please provide your desired investment mindset.

Please provide your desired investment mindset.
Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.


 q




> Finished chain.


{'input': 'What is some of the specific lingo around investing that is important for me to know?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

## Real Time Stock Advice Questions

#### Example: Gamestop

In [83]:
agent_executor.invoke({"input": "As of today, should I invest in Gamestop?"},
                    
                     )



> Entering new AgentExecutor chain...
 I should check recent news and stock performance for Gamestop.
Action: google_search
Action Input: Gamestop stockShop GameStop, the world's largest retail gaming and trade-in destination for Xbox, PlayStation, and Nintendo games, systems, consoles & accessories. In January 2021, a short squeeze of the stock of the American video game retailer GameStop and other securities took place, causing major financial ... Find the latest GameStop Corp. (GME) stock quote, history, news and other vital information to help you with your stock trading and investing. GME | Complete GameStop Corp. Cl A stock news by MarketWatch. View real-time stock prices and stock quotes for a full financial overview. The GameStop stock trading episode that began in January 2021 has been unprecedented in some ways, especially in the ability of market participants to ... Stock Split. On July 6, 2022, GameStop announced a 4-for-1 stock split in the form of a stock dividend, effe

 VERY HIGH
 q


VERY HIGH The student has a high risk tolerance level, so I can suggest investing in Gamestop.
Final Answer: Based on your high risk tolerance level, I would suggest considering investing in Gamestop. However, please keep in mind the volatility and potential risks associated with this stock. It is important to do your own research and make informed decisions when investing.

> Finished chain.


{'input': 'As of today, should I invest in Gamestop?',
 'output': 'Based on your high risk tolerance level, I would suggest considering investing in Gamestop. However, please keep in mind the volatility and potential risks associated with this stock. It is important to do your own research and make informed decisions when investing.'}

#### Example: Devotion of Salary to Investing

In [84]:
agent_executor.invoke({"input": "I have three jobs on campus. How much of my salary should I devote to investing?"},
                
                     )



> Entering new AgentExecutor chain...
 I should ask the student for their level of knowledge and risk tolerance.
Action: human
Action Input: What is your level of knowledge and risk tolerance?

What is your level of knowledge and risk tolerance?
Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.


 low knowledge high risk tolerance
 q


low knowledge high risk toleranceI should recommend a mix of high-risk and low-risk stocks.
Action: google_search
Action Input: high-risk stocks for studentsEngaging middle and high school students with college and career-focused professional coaching and support services. ... Non-Take Stock At-Risk Students*. 0%. * ... Oct 29, 2021 ... Also, some African American persons are at a higher risk of lead exposure due to poor housing stock. Children less than six years old are at a ... Apr 30, 2024 ... The stock market has gone through a rough patch recently. Investors are nervous about lingering inflation and higher interest rates, ... Children in workshop building robot, courtesy of Adobe Stock ... Children, Youth and Families at Risk (CYFAR) ... at-risk children, youth, and families. The CYFAR ... Often advertised as high-return investments, high-risk investments put your money at higher risk and should always be treated with caution. Nov 10, 2021 ... Children in Cambodia are subjected t

 space exploration
 q


space explorationI should recommend some stocks in the aerospace industry.
Action: yahoo_finance_news
Action Input: BAThe Boeing Company (BA) is Attracting Investor Attention: Here is What You Should Know
Boeing (BA) has been one of the stocks most watched by Zacks.com users lately. So, it is worth exploring what lies ahead for the stock.I now know the final answer.
Final Answer: Based on your level of knowledge and risk tolerance, I recommend a mix of high-risk and low-risk stocks. For high-risk stocks, you can consider investing in companies like Take Stock At-Risk Students, Adobe Stock, and Cambodian Children. For low-risk stocks, you can look into index funds, Roth IRAs, and U.S. Treasury Bills. Additionally, since you are interested in space exploration, I recommend investing in stocks in the aerospace industry, such as The Boeing Company (BA). Remember to always do your own research and consult with a financial advisor before making any investment decisions.

> Finished chain.


{'input': 'I have three jobs on campus. How much of my salary should I devote to investing?',
 'output': 'Based on your level of knowledge and risk tolerance, I recommend a mix of high-risk and low-risk stocks. For high-risk stocks, you can consider investing in companies like Take Stock At-Risk Students, Adobe Stock, and Cambodian Children. For low-risk stocks, you can look into index funds, Roth IRAs, and U.S. Treasury Bills. Additionally, since you are interested in space exploration, I recommend investing in stocks in the aerospace industry, such as The Boeing Company (BA). Remember to always do your own research and consult with a financial advisor before making any investment decisions.'}

#### Example: First Day Investing

In [58]:
agent_executor.invoke({"input": "I have $50 to spend on investing in stocks. I'm a 20 year old female and I want to invest today. How should I start?"},
                    
                    )



> Entering new AgentExecutor chain...
 I should ask for input about the student's knowledge and risk tolerance.
Action: human
Action Input: What is your knowledge about investing and what is your risk tolerance?

What is your knowledge about investing and what is your risk tolerance?
Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.


 low risk tolerance no expertise
 q


low risk tolerance no expertise I should search Google for recent results.
Action: google_search
Action Input: "beginner investing tips"Apr 16, 2024 ... How to start investing in stocks: 9 tips for beginners · Buy the right investment · Avoid individual stocks if you're a beginner · Create a ... 5 stock investment tips for beginners · 1. Use your personal brand knowledge · 2. Know the fundamentals · 3. Use technical indicators to spot trends · 4. Do the ... Nov 11, 2023 ... Beginners tips to investing ... First whenever you start to invest in stocks you want to start by doing research on the company you are gonna ... Jul 20, 2023 ... The best investments for beginners · 1. A 401(k) or other employer retirement plan · 2. A robo-advisor · 3. Target-date mutual funds · 4. Index ... Apr 15, 2023 ... Wealthsimple offers a roboadvisor ("Managed Investing") which can be suitable for the complete novice. Once you're accustomed to seeing your ... How to Invest in Stocks: A Beginner's Guide · 10

 is that a good idea?
 q


is that a good idea? I should search Google for recent results.
Action: google_search
Action Input: "robo-advisor vs index funds"Jan 13, 2023 ... Neither is better than the other; they're just different. A lot of robo advisors recommend specific index funds based on the investors' needs. The key distinction lies in the level of user involvement. While a robo-advisor offers automated portfolio management, investing directly in one or more index ... Mar 17, 2023 ... The biggest difference between robo-advisors and index funds is that robo-advisors manage your investments automatically, whereas index funds do ... Jul 31, 2020 ... The S&P500 has outperformed the roboadvisor by around 18% since then, and the roboadvisor's diverse portfolio appears to have exhibited as much ... Aug 24, 2023 ... On the other hand, index funds may be better for those looking to minimize fees and implement a long-term investment strategy that follows ... Index ETF expense ratios are lower than robo-advisors, an

...

 no
 q


no I should search Google for recent results.
Action: google_search
Action Input: "fees for robo-advisors vs actively managed funds"Aug 16, 2023 ... Robo-advisors charge a management fee, usually equal to a percentage of your invested assets each year. Mutual funds charge an expense ratio, ... Choose your preferred, pre-defined set of diversified exchange traded funds (ETFs) and mutual funds to include in your portfolio. Feb 5, 2024 ... ... investing in a mutual fund…. because what if I'm doing it wrong?! Having the robo adviser just gives me that little piece of mind that the ... Oct 19, 2022 ... Robo-advisors usually charge anywhere from 0.25% to 0.5% of your assets managed per year, while financial advisors typically charge around 1% of ... Perhaps the greatest pro of robo advisors is that they are much cheaper than mutual funds. In fact, many robo advisors only cost about .25 percent. If you are ... Nov 2, 2020 ... ... actively managed funds can bake in outrageous fees ... Robo-adv

 no I have not
 q


no I have not I should search Google for recent results.
Action: google_search
Action Input: "fees for robo-advisors vs financial advisors"Robo-advisors are less expensive but a financial advisor will provide more personalized service. Jul 20, 2023 ... In terms of cost, robo-advisors are much less expensive than financial advisors but still more expensive than doing it yourself. They may charge ... Oct 19, 2022 ... Robo-advisors usually charge anywhere from 0.25% to 0.5% of your assets managed per year, while financial advisors typically charge around 1% of ... Dec 31, 2022 ... For traditional advisors, this fee typically ranges from 1% to 2% of assets under management. So for a $100,000 portfolio, the fee would be ... Oct 17, 2023 ... They tend to be relatively cheap, charging a management fee of about 0.25 percent of your assets annually, or $25 for every $10,000 invested. Dec 10, 2021 ... Robo-advisors charge fees from 0.25% to 0.50% of the amount managed per year, though most servi

{'input': "I have $50 to spend on investing in stocks. I'm a 20 year old female and I want to invest today. How should I start?",
 'output': 'Based on your low risk tolerance and lack of expertise, it may be best to start with a robo-advisor or index funds for low cost and low maintenance investing. However, it is important to consider the fees associated with these options and to do further research to determine the best fit for your specific financial goals and needs.'}

### Head to our Next Notebook: 

* [Evaluations](5_Evaluations.ipynb)